**1. Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:**

* InvoiceNo — номер транзакции
* StockCode — код товара
* Description — описание товара
* Quantity — количество единиц товара, добавленных в заказ
* InvoiceDate — дата транзакции 
* UnitPrice — цена за единицу товара
* CustomerID — id клиента
* Country — страна, где проживает клиент

**В данной задаче ссылка для считывания данных лежит в переменной path_to_file.
Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.**

In [7]:
import pandas as pd

retail = pd.read_csv('data.csv',encoding='ISO-8859-1') 
retail_columns = retail.columns

print(retail.head())
print(retail_columns)

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')


**2. Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.**

Вам понадобится len(название_колонки) и drop_duplicates()

In [9]:
test_full = len(retail)
test_once = len(retail.drop_duplicates())
print(test_full)
print(test_once)
print(test_full - test_once)

541909
536641
5268


**3. Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled).**

**Сколько всего транзакций отменили пользователи? Каждый отдельный тип продукта считайте за отдельную транзакцию - уникальные номера считать не надо.**

Прежде чем начать, попробуйте метод retail.InvoiceNo.str[0]

Далее используйте метод .unique()

И сделайте выбоку по retail.loc[retail.InvoiceNo.str[0] == 'C'].shape

In [14]:
print(retail.loc[retail.InvoiceNo.str[0] == 'C'].shape[0]) 
//Взято общще число отменённых заказов


9288


**5.Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).**

Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

**Подсказка:**
1. Начните с выборки retail.Country == 'Germany', группировки по 'CustomerID' и агрегации total_invoices=('InvoiceNo', 'nunique')
2. Далее из полученой выборки из предыдущего шага выподните метод: название_выборки.столбец.quantile(0.8)
3. Запишите полученные id пользователей в germany_top

In [28]:
retail_germany = retail[retail['Country'] == 'Germany']

customer_orders = retail_germany.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
customer_orders.columns = ['CustomerID', 'InvoiceCount']

percentile_80 = customer_orders['InvoiceCount'].quantile(0.8)

top_customers = customer_orders[customer_orders['InvoiceCount'] > percentile_80]

germany_top = top_customers['CustomerID'].unique()

print(germany_top)

[12471. 12472. 12474. 12476. 12481. 12500. 12569. 12600. 12619. 12621.
 12626. 12647. 12662. 12705. 12708. 12709. 12712. 12720.]


**6. Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.**

Датафрейм записан в retail, нужные id пользователей – в germany_top.

**Подсказка**
Попробуйте использовать top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]

In [30]:
top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]
print(top_retail_germany)

       InvoiceNo StockCode                          Description  Quantity  \
1109      536527     22809              SET OF 6 T-LIGHTS SANTA         6   
1110      536527     84347  ROTATING SILVER ANGELS T-LIGHT HLDR         6   
1111      536527     84945   MULTI COLOUR SILVER T-LIGHT HOLDER        12   
1112      536527     22242        5 HOOK HANGER MAGIC TOADSTOOL        12   
1113      536527     22244           3 HOOK HANGER MAGIC GARDEN        12   
...          ...       ...                                  ...       ...   
541726    581570     22139     RETROSPOT TEA SET CERAMIC 11 PC          3   
541727    581570     23077                  DOUGHNUT LIP GLOSS         20   
541728    581570     20750             RED RETROSPOT MINI CASES         2   
541729    581570     22505            MEMO BOARD COTTAGE DESIGN         4   
541730    581570      POST                              POSTAGE         1   

            InvoiceDate  UnitPrice  CustomerID  Country  
1109    12/1/2010

**7. Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST?** 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [35]:
# Исключаем товар с кодом POST
filtered_data = top_retail_germany[top_retail_germany['StockCode'] != 'POST']

# Группируем по коду товара и считаем уникальные заказы
product_stats = filtered_data.groupby('StockCode')['InvoiceNo'].nunique().reset_index()
product_stats.columns = ['StockCode', 'UniqueOrders']

# Сортируем и находим самый популярный товар
top_product = product_stats.sort_values('UniqueOrders', ascending=False).iloc[0]

# Результат
print(f"Самый популярный товар: {top_product['StockCode']}")
print(f"Количество заказов: {top_product['UniqueOrders']}")


Самый популярный товар: 22326
Количество заказов: 62


**8. Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.**

Отфильтрованные данные записаны в retail.

**Подсказка**
retail['новый_столбец'] = retail.старый_столбец_1 * retail.старый_столбец_2

In [42]:
retail.loc[:, 'Revenue'] = retail['Quantity'] * retail['UnitPrice']

print(retail['Revenue'])

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
541904    10.20
541905    12.60
541906    16.60
541907    16.60
541908    14.85
Name: Revenue, Length: 531285, dtype: float64
